In [9]:
from keras.models import load_model
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.applications import imagenet_utils
from CAM import CAM
import numpy as np
import argparse
import cv2
import imutils

In [4]:
model =  load_model('../models/vgg16-pretrained-rmsprop.model/')

# Check its architecture
model.summary()


Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None, None, 3)]   0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)  

In [5]:
IMAGE_PATH = "./4_100.jpg"
orig = cv2.imread(IMAGE_PATH)
resized = cv2.resize(orig, (224, 224))
# load the input image from disk (in Keras/TensorFlow format) and
# preprocess it
image = load_img(IMAGE_PATH, target_size=(224, 224))
image = img_to_array(image)
image = np.expand_dims(image, axis=0)
image = imagenet_utils.preprocess_input(image)

In [6]:
preds = model.predict(image)
i = np.argmax(preds[0])
print("CLASS", i)
# decode the ImageNet predictions to obtain the human-readable label
# decoded = imagenet_utils.decode_predictions(preds)
# print(decoded)
# (imagenetID, label, prob) = decoded[0][0]
# label = "{}: {:.2f}%".format(label, prob * 100)
# print("[INFO] {}".format(label))

CLASS 14


In [7]:
cam = CAM(model, i)
heatmap = cam.compute_heatmap(image)
print(heatmap, heatmap.shape, image.shape)
# resize the resulting heatmap to the original input image dimensions
# and then overlay heatmap on top of the image

heatmap = cv2.resize(heatmap, (orig.shape[1], orig.shape[0]))
(heatmap, output) = cam.overlay_heatmap(heatmap, orig, alpha=0.5)

[[ 7  7  7 ...  3  3  3]
 [ 7  7  7 ...  3  3  3]
 [ 7  7  7 ...  3  3  3]
 ...
 [ 8  8  8 ... 13 13 13]
 [ 8  8  8 ... 13 13 13]
 [ 8  8  8 ... 13 13 13]] (224, 224) (1, 224, 224, 3)


In [10]:
# draw the predicted label on the output image
# cv2.rectangle(output, (0, 0), (340, 40), (0, 0, 0), -1)
# cv2.putText(output, 'result', (10, 25), cv2.FONT_HERSHEY_SIMPLEX,0.8, (255, 255, 255), 2)
# display the original image and resulting heatmap and output image
# to our screen
output = np.vstack([orig, heatmap, output])
output = imutils.resize(output, height=700)
cv2.imshow("Output", output)
cv2.waitKey(0)

-1